In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from R125_R32_PR3 import configuration

# Load data from csv
data = pd.read_csv('R32R125.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(1669700)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.2688)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.7312)
    
    m.fs.properties.PR_kappa['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa['R32', 'R125'].fix(-0.05226)
    m.fs.properties.PR_kappa['R125', 'R32'].fix(0.00179)

    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
#     m.fs.state_block.pressure.unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
#     m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_comp["R125"].unfix()
    m.fs.state_block.mole_frac_comp["R32"].unfix()
#     m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"]))
#     m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))
    m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].unfix()
    
#     m.fs.kappas = Constraint(expr=m.fs.properties.PR_kappa['R32', 'R125'] == 
#                     m.fs.properties.PR_kappa['R125', 'R32'])
#     m.fs.properties.PR_kappa['R32', 'R125'] = Expression(expr=value(m.fs.properties.PR_kappa['R125', 'R32']))
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa['R32', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa['R32', 'R125'].setub(3)

    m.fs.properties.PR_kappa['R125', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa['R125', 'R32'].setub(3)

    # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 1669700, "x_R32":0.7312, 
             "x_R125":0.2688, "y_R125":0.2501, "y_R32":0.7499}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

# Check that degrees of freedom is 0
# assert degrees_of_freedom(m) == 0

2021-03-12 09:53:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 1


In [7]:
variable_name = ["fs.properties.PR_kappa['R32', 'R125']",
                 "fs.properties.PR_kappa['R125', 'R32']"]

In [8]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-7

# def SSE(m, data):
#     expr = (
#             (float(data["y_R125"]) -
#              m.fs.state_block.mole_frac_phase_comp["Vap", "R125"])**2
# #             + (float(data["x_R125"]) -
# #              m.fs.state_block.mole_frac_phase_comp["Liq", "R125"])**2
#            )
    
#     return expr*1e4


In [9]:
solver_options = {'tol': 1e-4}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, solver_options)
# solver_options = {'tol': 1e-6, 'max_iter':100}

obj_value, parameters = pest.theta_est()

2021-03-12 09:53:45 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-12 09:53:45 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-12 09:53:45 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-12 09:53:45 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
2021-03-12 09:53:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-12 09:53:47 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-12 0

2021-03-12 09:54:18 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-12 09:54:18 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-12 09:54:18 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
2021-03-12 09:54:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-12 09:54:23 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-12 09:54:23 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-12 0

2021-03-12 09:54:47 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-12 09:54:47 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
2021-03-12 09:54:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-12 09:54:49 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-12 09:54:49 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-12 09:54:49 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021

2021-03-12 09:55:08 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
2021-03-12 09:55:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-12 09:55:11 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-12 09:55:11 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-12 09:55:11 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-12 09:55:11 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
202

2021-03-12 09:55:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-12 09:55:32 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-12 09:55:32 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-12 09:55:32 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-12 09:55:32 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
2021-03-12 09:55:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-12 09:55:34 [WARNING] idaes.generic_models.properties

...

2464r 5.1497321e+04 1.76e+01 2.96e+06  -0.2 1.77e+00   2.3 1.65e-01 3.77e-02f  1
2465r 5.1497321e+04 1.76e+01 2.95e+06  -0.2 2.22e-01   5.5 3.21e-03 4.70e-03h  1
2466r 5.1497321e+04 1.75e+01 2.95e+06  -0.2 2.97e-01   5.0 1.75e-03 5.98e-03H  1
2467r 5.1497321e+04 1.75e+01 3.04e+06  -0.2 3.67e-01   4.5 5.94e-03 3.06e-02H  1
2468r 5.1497321e+04 1.75e+01 3.00e+06  -0.2 2.63e-01   4.9 2.33e-02 3.29e-02h  1
2469r 5.1497321e+04 1.75e+01 3.00e+06  -0.2 2.01e-01   7.2 1.47e-03 1.03e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2470r 5.1497321e+04 1.75e+01 2.98e+06  -0.2 2.19e-01   7.6 5.64e-03 7.62e-04h  1
2471r 5.1497321e+04 1.75e+01 2.92e+06  -0.2 1.71e+00   7.1 1.63e-02 4.63e-03h  1
2472r 5.1497321e+04 1.75e+01 2.86e+06  -0.2 2.23e-01   6.6 2.44e-02 4.32e-02h  1
2473r 5.1497321e+04 1.75e+01 2.71e+06  -0.2 1.86e-01   7.1 4.33e-02 1.15e-02h  1
2474r 5.1497321e+04 1.75e+01 2.72e+06  -0.2 2.69e-01   6.6 6.47e-03 1.64e-02h  1
2475r 5.1497321e+04 1.75e+01

2602r 5.1497321e+04 1.76e+01 7.47e+08  -0.2 5.01e-02  10.5 3.05e-05 2.40e-03f  2
2603r 5.1497321e+04 1.76e+01 7.75e+09  -0.2 4.04e-02  11.0 1.39e-02 1.30e-03h  1
2604r 5.1497321e+04 1.76e+01 3.55e+10  -0.2 3.49e-02  12.3 4.65e-05 2.40e-03H  1
2605r 5.1497321e+04 1.76e+01 4.21e+10  -0.2 3.78e-02  12.7 6.67e-05 4.36e-04h  1
2606r 5.1497321e+04 1.76e+01 1.16e+11  -0.2 6.82e-01  12.2 1.46e-06 1.82e-05f  4
2607r 5.1497321e+04 1.76e+01 1.84e+11  -0.2 3.41e-02  13.6 3.47e-05 2.06e-04H  1
2608r 5.1497321e+04 1.76e+01 1.84e+11  -0.2 5.61e-02  13.1 0.00e+00 3.63e-07R  3
2609r 5.1497321e+04 1.76e+01 8.72e+05  -0.2 3.55e-11  13.5 9.90e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2610r 5.1497321e+04 1.76e+01 7.30e+03  -0.2 9.84e-11  13.0 9.92e-01 1.00e+00h  1
2611r 5.1497321e+04 1.76e+01 1.10e+03  -0.2 2.95e-10  12.6 1.00e+00 1.00e+00f  1
2612r 5.1497321e+04 1.76e+01 1.10e+03  -0.2 8.85e-10  12.1 1.00e+00 1.00e+00f  1
2613r 5.1497321e+04 1.76e+01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2740r 5.1497321e+04 1.74e+01 8.85e+04  -0.2 1.22e-01   4.3 9.17e-02 1.23e-01f  1
2741r 5.1497321e+04 1.74e+01 5.54e+04  -0.2 1.66e-01   3.8 1.00e+00 2.85e-01f  1
2742r 5.1497321e+04 1.74e+01 8.63e+04  -0.2 4.43e-01   3.4 5.54e-02 7.26e-02f  1
2743r 5.1497321e+04 1.75e+01 8.15e+04  -0.2 4.59e-01   3.8 2.56e-01 1.91e-01f  1
2744r 5.1497321e+04 1.75e+01 1.24e+05  -0.2 5.07e-01   3.3 1.21e-01 8.46e-02f  1
2745r 5.1497321e+04 1.75e+01 1.22e+05  -0.2 3.15e-01   3.7 3.20e-02 1.19e-02f  1
2746r 5.1497321e+04 1.75e+01 1.30e+05  -0.2 3.19e-01   4.2 1.98e-02 7.67e-02h  1
2747r 5.1497321e+04 1.75e+01 1.29e+05  -0.2 2.10e-01   3.7 3.55e-01 1.10e-01f  1
2748r 5.1497321e+04 1.75e+01 1.63e+05  -0.2 2.27e-01   4.1 4.43e-03 2.93e-02f  1
2749r 5.1497321e+04 1.75e+01 1.32e+05  -0.2 6.54e-01   3.6 1.20e-01 5.11e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2750r 5.1497321e+04 1.75e+01

2832r 5.1497321e+04 1.75e+01 9.24e+05  -0.2 1.87e+00   2.9 1.25e-01 1.92e-02f  1
2833r 5.1497321e+04 1.75e+01 2.44e+06  -0.2 4.15e+00   2.4 2.10e-01 3.72e-02f  1
2834r 5.1497321e+04 1.75e+01 4.26e+06  -0.2 9.58e-01   1.9 3.67e-01 8.75e-03f  1
2835r 5.1497321e+04 1.74e+01 3.03e+06  -0.2 8.66e-01   1.4 1.38e-01 1.77e-01f  1
2836r 5.1497321e+04 1.74e+01 2.97e+06  -0.2 5.33e-01   4.6 1.96e-02 1.92e-02f  1
2837r 5.1497321e+04 1.74e+01 2.94e+06  -0.2 1.19e+00   5.0 1.64e-02 3.48e-02f  1
2838r 5.1497321e+04 1.74e+01 1.91e+06  -0.2 2.03e-01   4.5 2.81e-01 1.71e-02h  1
2839r 5.1497321e+04 1.74e+01 1.91e+06  -0.2 2.86e+00   4.0 2.35e-04 3.74e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2840r 5.1497321e+04 1.74e+01 1.46e+06  -0.2 2.42e-01   3.6 2.19e-01 1.93e-03h  1
2841r 5.1497321e+04 1.74e+01 1.33e+06  -0.2 2.64e-01   3.1 3.03e-03 1.51e-01f  1
2842r 5.1497321e+04 1.74e+01 1.06e+06  -0.2 6.08e-01   2.6 5.77e-03 8.72e-02f  1
2843r 5.1497321e+04 1.74e+01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2970r 5.1497321e+04 1.75e+01 3.62e+05  -0.2 2.67e-01   4.7 3.21e-02 3.83e-02h  1
2971r 5.1497321e+04 1.75e+01 7.29e+05  -0.2 4.90e-01   4.3 7.43e-02 8.10e-03f  1
2972r 5.1497321e+04 1.75e+01 6.39e+05  -0.2 2.35e-01   4.7 5.48e-02 7.48e-02h  2
2973r 5.1497321e+04 1.75e+01 3.91e+05  -0.2 1.74e-02   5.1 3.74e-01 1.38e-02h  5
2974r 5.1497321e+04 1.75e+01 5.10e+04  -0.2 5.01e-02   4.6 8.79e-01 2.39e-01f  1
2975r 5.1497321e+04 1.75e+01 6.71e+04  -0.2 1.01e-01   4.2 1.00e+00 4.96e-01f  1
2976r 5.1497321e+04 1.75e+01 9.60e+04  -0.2 2.64e-01   3.7 4.76e-02 6.86e-02f  1
2977r 5.1497321e+04 1.75e+01 7.02e+05  -0.2 6.08e-01   3.2 3.63e-01 3.35e-02f  1
2978r 5.1497321e+04 1.75e+01 7.16e+05  -0.2 8.25e-01   3.6 1.74e-02 2.10e-02f  1
2979r 5.1497321e+04 1.75e+01 9.60e+05  -0.2 6.76e-01   3.2 1.03e-01 2.23e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2980r 5.1497321e+04 1.75e+01

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 51497.320894

The values for the parameters are as follows:
fs.properties.PR_kappa[R125,R32] = -0.13253571251843493
fs.properties.PR_kappa[R32,R125] = 0.25753084774332363


In [11]:
pest.ef_instance.display()

Model MASTER

  Variables:
    MASTER_BLEND_VAR_RootNode : Size=2, Index=MASTER_BLEND_VAR_RootNode_index
        Key                              : Lower : Value               : Upper : Fixed : Stale : Domain
        fs.properties.PR_kappa[R125,R32] :  None : -0.1515971883726917 :  None : False : False :  Reals
        fs.properties.PR_kappa[R32,R125] :  None : 0.26476704250404726 :  None : False : False :  Reals

  Objectives:
    MASTER : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 51497.320893939395

  Constraints:
    MASTER_BLEND_CONSTRAINT_RootNode : Size=132
        Key : Lower : Body                    : Upper
          1 :   0.0 :  2.2609484055385876e-05 :   0.0
          2 :   0.0 :   3.561998465789751e-05 :   0.0
          3 :   0.0 :  0.00013219700210778673 :   0.0
          4 :   0.0 :  0.00017223789812809143 :   0.0
          5 :   0.0 :   0.0001716224913708153 :   0.0
          6 :   0.0 :  -9.296914096532105e-05 :   0.0
        

                    ('Vap', 'Liq', 'R125') :   0.0 : 5.753438024680424e-05 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 :  9.77847807348553e-06 :   0.0
                eq_mole_frac_tbub : Size=1
                    Key            : Lower  : Body               : Upper
                    ('Vap', 'Liq') : 1000.0 : 1000.0000063443925 : 1000.0
                _teq_constraint_Vap_Liq : Size=1
                    Key  : Lower : Body                  : Upper
                    None :   0.0 : 2.754008227157101e-05 :   0.0
                eq_temperature_dew : Size=2
                    Key                    : Lower : Body                   : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 :   0.008643265027670655 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : 0.00019535049004204552 :   0.0
                eq_mole_frac_tdew : Size=1
                    Key            : Lower  : Body              : Upper
                    ('Vap', 'Liq') : 1000.0 : 999.9998

                    ('Vap', 'Liq', 'R125') :     0 :   0.822125028338229 :  None : False : False :  Reals
                     ('Vap', 'Liq', 'R32') :     0 : 0.17787497380075684 :  None : False : False :  Reals
                temperature_dew : Dew point temperature
                    Size=1, Index=Experiment1.fs.properties._pe_pairs
                    Key            : Lower : Value             : Upper : Fixed : Stale : Domain
                    ('Vap', 'Liq') :    10 : 301.5826638449224 :  1000 : False : False :  Reals
                _mole_frac_tdew : Liquid mole fractions at dew temperature
                    Size=2, Index=Experiment1.fs.state_block._mole_frac_tdew_index
                    Key                    : Lower : Value               : Upper : Fixed : Stale : Domain
                    ('Vap', 'Liq', 'R125') :     0 :  0.9843857280824667 :  None : False : False :  Reals
                     ('Vap', 'Liq', 'R32') :     0 : 0.01561427090925172 :  None : False : False :  

                    ('Vap', 'Liq', 'R125') :   0.0 : -9.015726040395577e-05 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : -4.573612328417909e-06 :   0.0
                eq_mole_frac_tdew : Size=1
                    Key            : Lower  : Body               : Upper
                    ('Vap', 'Liq') : 1000.0 : 1000.0000099883646 : 1000.0
                equilibrium_constraint : Size=2
                    Key                    : Lower : Body                    : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 :   -0.002914449403653485 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : -0.00019516799768726223 :   0.0
    Block Experiment3
    
      Variables:
        None
    
      Objectives:
        None
    
      Constraints:
        None
    
      Blocks:
        Block Experiment3.fs
        
          Variables:
            None
        
          Objectives:
            None
        
          Constraints:
            None
        
        

                    pressure_sat_comp_coeff_B : Antoine B coefficient for calculating Psat
                        Size=1, Index=None
                        Key  : Lower : Value     : Upper : Fixed : Stale : Domain
                        None :  None : 1017.2097 :  None :  True :  True :  Reals
                    pressure_sat_comp_coeff_C : Antoine C coefficient for calculating Psat
                        Size=1, Index=None
                        Key  : Lower : Value      : Upper : Fixed : Stale : Domain
                        None :  None : 8.3042e-05 :  None :  True :  True :  Reals
                
                  Objectives:
                    None
                
                  Constraints:
                    None
            Block Experiment3.fs.state_block
            
              Variables:
                flow_mol :  Total molar flowrate
                    Size=1, Index=None
                    Key  : Lower : Value : Upper : Fixed : Stale : Domain
            

...

                    ('Vap', 'Liq', 'R125') :   0.0 : -0.007850215022043638 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 :  -0.09282063242093841 :   0.0
    Block Experiment61
    
      Variables:
        None
    
      Objectives:
        None
    
      Constraints:
        None
    
      Blocks:
        Block Experiment61.fs
        
          Variables:
            None
        
          Objectives:
            None
        
          Constraints:
            None
        
          Blocks:
            Block Experiment61.fs.properties
            
              Variables:
                PR_kappa : PR binary interaction parameters
                    Size=4, Index=Experiment61.fs.properties.PR_kappa_index
                    Key              : Lower : Value                : Upper : Fixed : Stale : Domain
                    ('R125', 'R125') :  None :                    0 :  None :  True :  True :  Reals
                     ('R125', 'R32') :    -3 : -0.152342840924

                    ('Vap', 'Liq', 'R125') :   0.0 : -5.688887621957406e-05 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 :   -0.04317517186037251 :   0.0
                eq_mole_frac_tbub : Size=1
                    Key            : Lower  : Body              : Upper
                    ('Vap', 'Liq') : 1000.0 : 999.9998406846953 : 1000.0
                _teq_constraint_Vap_Liq : Size=1
                    Key  : Lower : Body                   : Upper
                    None :   0.0 : -2.396037473317847e-05 :   0.0
                eq_temperature_dew : Size=2
                    Key                    : Lower : Body                   : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 : -0.0007351354486285766 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 :  -0.000734915035300987 :   0.0
                eq_mole_frac_tdew : Size=1
                    Key            : Lower  : Body              : Upper
                    ('Vap', 'Liq') : 1000.0 : 1000.0

                    ('Vap', 'Liq', 'R125') :   0.0 : 3.4946397466484314e-05 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : -0.0007534327756835157 :   0.0
                eq_mole_frac_tbub : Size=1
                    Key            : Lower  : Body              : Upper
                    ('Vap', 'Liq') : 1000.0 : 999.9999278262704 : 1000.0
                _teq_constraint_Vap_Liq : Size=1
                    Key  : Lower : Body                    : Upper
                    None :   0.0 : -2.1258864535411703e-05 :   0.0
                eq_temperature_dew : Size=2
                    Key                    : Lower : Body                   : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 : -0.0003940659127423807 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : -0.0003044824797281409 :   0.0
                eq_mole_frac_tdew : Size=1
                    Key            : Lower  : Body               : Upper
                    ('Vap', 'Liq') : 1000.0 : 100

                        Size=1, Index=None
                        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
                        None :  None : -269.0 :  None :  True :  True :  Reals
                    pressure_sat_comp_coeff_A : Antoine A coefficient for calculating Psat
                        Size=1, Index=None
                        Key  : Lower : Value   : Upper : Fixed : Stale : Domain
                        None :  None : 4.55069 :  None :  True :  True :  Reals
                    pressure_sat_comp_coeff_B : Antoine B coefficient for calculating Psat
                        Size=1, Index=None
                        Key  : Lower : Value     : Upper : Fixed : Stale : Domain
                        None :  None : 1017.2097 :  None :  True :  True :  Reals
                    pressure_sat_comp_coeff_C : Antoine C coefficient for calculating Psat
                        Size=1, Index=None
                        Key  : Lower : Value      : Upper : Fixed : Stale

                _t1_constraint_Vap_Liq : Size=1
                    Key  : Lower : Body                    : Upper
                    None :   0.0 : -1.4711645235365722e-09 :   0.0
                eq_temperature_bubble : Size=2
                    Key                    : Lower : Body                 : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 :  0.00091293029021422 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : 0.005856736412292396 :   0.0
                eq_mole_frac_tbub : Size=1
                    Key            : Lower  : Body               : Upper
                    ('Vap', 'Liq') : 1000.0 : 1000.0001384657389 : 1000.0
                _teq_constraint_Vap_Liq : Size=1
                    Key  : Lower : Body                   : Upper
                    None :   0.0 : 1.1637293653166125e-05 :   0.0
                eq_temperature_dew : Size=2
                    Key                    : Lower : Body                   : Upper
                    ('Vap'

In [12]:
print(large_residuals_set(pest.ef_instance))

ComponentSet(['MASTER_BLEND_CONSTRAINT_RootNode[1] (id=2631102657352)', 'MASTER_BLEND_CONSTRAINT_RootNode[2] (id=2631102657448)', 'MASTER_BLEND_CONSTRAINT_RootNode[3] (id=2631102657640)', 'MASTER_BLEND_CONSTRAINT_RootNode[4] (id=2631102657736)', 'MASTER_BLEND_CONSTRAINT_RootNode[5] (id=2631102657544)', 'MASTER_BLEND_CONSTRAINT_RootNode[6] (id=2631102658120)', 'MASTER_BLEND_CONSTRAINT_RootNode[7] (id=2631102659080)', 'MASTER_BLEND_CONSTRAINT_RootNode[8] (id=2631102658984)', 'MASTER_BLEND_CONSTRAINT_RootNode[9] (id=2631102658792)', 'MASTER_BLEND_CONSTRAINT_RootNode[10] (id=2631102658888)', 'MASTER_BLEND_CONSTRAINT_RootNode[11] (id=2631102658696)', 'MASTER_BLEND_CONSTRAINT_RootNode[12] (id=2631102658600)', 'MASTER_BLEND_CONSTRAINT_RootNode[13] (id=2631102658216)', 'MASTER_BLEND_CONSTRAINT_RootNode[14] (id=2631102657928)', 'MASTER_BLEND_CONSTRAINT_RootNode[15] (id=2631102657832)', 'MASTER_BLEND_CONSTRAINT_RootNode[16] (id=2631102659752)', 'MASTER_BLEND_CONSTRAINT_RootNode[17] (id=263110265